In [1]:
def ext_scalars(k, n):
    q = k.order()
    assert gcd(n, q) == 1, "Only multiplicative H90, please!"
    
    phi = cyclotomic_polynomial(n, 'z').change_ring(k)
    
    K = k.extension(n, 'x')
    A = K['z']
    h = A(phi.factor()[0][0])
    
    return K, A.quo(h, 'z')

In [2]:
def frob_l(a, i=1):
    A = a.parent()
    q = a.base_ring().base_ring().order()
    return A(list(c^(q^i) for c in a))

In [3]:
def frob_r(a, i=1):
    A = a.parent()
    z = A.gen()
    q = a.base_ring().base_ring().order()
    return sum(c*z^(j*q^i) for j, c in enumerate(a))

In [4]:
def is_h90(a, zeta):
    return frob_l(a) == zeta * a

In [5]:
def solve_h90(A):
    x = A(1)
    n = A.base_ring().degree()
    z = A.gen()
    while not is_h90(x, z):
        x = A.random_element()
        x = sum(frob_l(x, i)*z^-i for i in range(n))
    return x

In [6]:
K, A = ext_scalars(GF(7), 9)  # 1 = r₁ < r₂ = 2
a = A.random_element()
h = solve_h90(A)
u = h*frob_r(h)*frob_r(h, 2)
v = rnorm(h, 3)
is_h90(u, A.gen()^3), frob_l(u, 3) == u, u == v

NameError: name 'rnorm' is not defined

In [29]:
K, A = ext_scalars(GF(19), 9)  # 1 = r₁ = r₂
a = A.random_element()
h = solve_h90(A)
u = h*frob_r(h)*frob_r(h, 2)
v = rnorm(h, 3)
is_h90(u, A.gen()^3), frob_l(u, 3) == u, u == v

9 3 1 1 1


(True, True, False)

In [41]:
K, A = ext_scalars(GF(7), 15)
a = A.random_element()
h = solve_h90(A)
u = h*frob_r(h, 1)*frob_r(h, 2)*frob_r(h, 3)
v = rnorm(h, 3)
is_h90(u, A.gen()^10), frob_r(u, 3) == u, u == v

15 3 4 1 4


(True, True, True)

In [46]:
K, A = ext_scalars(GF(7), 15)
a = A.random_element()
h = solve_h90(A)
u = h*frob_r(h, 4)*frob_r(h, 8)
v = rnorm(h, 5)
is_h90(u, A.gen()^3), frob_r(u, 4) == u, is_h90(v, A.gen()^3), frob_r(v, 4) == v, u == v, u == h^3

15 5 4 4 1


(True, True, False, True, False, True)

In [100]:
K, A = ext_scalars(GF(19), 15)
a = A.random_element()
h = solve_h90(A)
u = h*frob_r(h, 1)
is_h90(u, A.gen()^5), frob_l(u, 3) == u

(False, False)

# More tests

In [19]:
K, A = ext_scalars(GF(19), 21)
a = A.random_element()
h = solve_h90(A)
#u = h*frob_r(h, 1)*frob_r(h, 2)*frob_r(h, 3)*frob_r(h, 4)*frob_r(h, 5)
u = h^7
v = rnorm(h, 3)
is_h90(u, A.gen()^7), frob_r(u, 21) == u, u == v

(True, False, False)

In [17]:
cyclo_deg(GF(19), 7)

6

In [6]:
def cyclo_deg(k, n):
    return Zmod(n)(k.characteristic()).multiplicative_order()

In [91]:
def rnorm(a, m):
    A = a.parent()
    n = A.base_ring().degree()
    r = A.degree()
    q = A.base_ring().base_ring().order()
    s = cyclo_deg(GF(q), m)
    d = n // m
    
#    print n, m, r, s, d
    return product(frob_r(a, s*j) for j in range(d))

In [19]:
def toto(k, m, n):
    q = k.order()
    r = cyclo_deg(k, m)
    s = cyclo_deg(k, n)
    t = (q^s-1)//(q^r-1)
    return t, gcd(t, n), n//m

def change_basis(elem, basis, d2):
    
    k = elem.base_ring()
    d1 = elem.parent().degree()
    A = MatrixSpace(k, d1, d2)()
    
    for i in range(d2):
        L = (basis^i).list()
        for j in range(d1):
            A[j,i] = L[j]
            
    S2 = MatrixSpace(k, d1, 1)
    B = S2(elem.list())
    X = A.solve_right(B)
    
    return X[0,0]

In [37]:
for j in range(1, 8):
    print toto(GF(29), 7^j, 7^(j+1))

(7, 7)
(7, 7)
(7, 7)
(7, 7)
(7, 7)
(7, 7)
(7, 7)


In [16]:
for p in primes(5, 100):
    print toto(GF(p), 3^3, 3^4)

(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)
(3, 3, 3)


In [15]:
primes?

# An $\ell$-adic tower

In [10]:
p = 17
l = 3
cyclo_deg(GF(p), l^2)

2

In [11]:
k = GF(p)
R.<x> = k[]

In [57]:
f = factor(R(cyclotomic_polynomial(l^2))); f

(x^2 + 3*x + 1) * (x^2 + 4*x + 1) * (x^2 + 10*x + 1)

In [25]:
factor(R(cyclotomic_polynomial(l^3)))

(x^6 + 3*x^3 + 1) * (x^6 + 4*x^3 + 1) * (x^6 + 10*x^3 + 1)

In [29]:
g = gcd(f.substitute(x^l), R(cyclotomic_polynomial(l^3))); g

x^6 + 3*x^3 + 1

In [30]:
K.<z> = k.extension(g)

In [31]:
(z^l).minpoly()

x^2 + 3*x + 1

In [39]:
(z^((p^6-1)/(p^2-1))).minpoly()

x^2 + 3*x + 1

In [40]:
(z^((p^6-1)/(p^2-1))) == (z^l)

True

In [12]:
K, A = ext_scalars(GF(p), 3^2)

In [16]:
A.modulus().parent()

Univariate Polynomial Ring in z over Finite Field in x of size 17^9

In [67]:
g = gcd(A.modulus().substitute(x^l), R(cyclotomic_polynomial(l^3))); g

x^6 + 3*x^3 + 1

In [68]:
h = solve_h90(A); h

(15*x^8 + 2*x^7 + 9*x^6 + 11*x^5 + 8*x^4 + 16*x^3 + 14*x^2 + 10*x + 8)*z + 9*x^8 + 14*x^7 + 5*x^6 + 14*x^5 + 12*x^3 + 4*x^2 + 8

Finite Field in z4 of size 17^4

In [78]:
k3 = k.extension(l^3)

In [83]:
R3 = k3['x']

In [89]:
g = gcd(A.modulus().change_ring(R3).substitute(R3.gen()^l), R3(cyclotomic_polynomial(l^3))); g

x^6 + 3*x^3 + 1

In [92]:
A3 = R3.quo(g, 'z')

In [94]:
h3 = solve_h90(A3); h3

(9*z27^26 + 12*z27^25 + 12*z27^24 + 9*z27^23 + 4*z27^22 + 8*z27^21 + 14*z27^20 + 15*z27^19 + 7*z27^18 + z27^17 + 13*z27^16 + 9*z27^15 + 5*z27^14 + 6*z27^13 + 15*z27^12 + 8*z27^11 + 4*z27^10 + 5*z27^9 + 10*z27^8 + 13*z27^7 + 6*z27^5 + 10*z27^4 + 2*z27^3 + 10*z27^2 + 12*z27 + 8)*z^5 + (14*z27^26 + 12*z27^25 + 6*z27^24 + 16*z27^23 + 5*z27^22 + 9*z27^21 + 2*z27^20 + 7*z27^19 + 4*z27^18 + 4*z27^17 + 5*z27^16 + 13*z27^15 + z27^14 + 5*z27^13 + 2*z27^12 + 9*z27^11 + 9*z27^10 + 15*z27^9 + 2*z27^8 + 8*z27^6 + 9*z27^5 + 10*z27^4 + 16*z27^3 + 10*z27^2 + 5*z27 + 9)*z^4 + (z27^26 + z27^25 + 6*z27^24 + 6*z27^23 + 9*z27^22 + 5*z27^21 + 7*z27^20 + 7*z27^19 + 16*z27^18 + 7*z27^17 + z27^16 + 11*z27^15 + 10*z27^13 + 9*z27^12 + 13*z27^11 + 11*z27^10 + 12*z27^9 + 3*z27^8 + z27^7 + 5*z27^6 + 15*z27^5 + 7*z27^4 + 13*z27^3 + 16*z27^2 + 4*z27 + 16)*z^3 + (2*z27^26 + 7*z27^25 + z27^24 + 7*z27^23 + 6*z27^22 + 9*z27^21 + 8*z27^20 + 4*z27^19 + 15*z27^18 + 8*z27^17 + z27^16 + 5*z27^15 + 2*z27^14 + 16*z27^13 + 12*z27

In [99]:
i3 = rnorm(h3, l^2); i3

27 9 6 2 3


(12*z27^26 + 13*z27^25 + 7*z27^24 + 11*z27^23 + 10*z27^22 + 7*z27^21 + 13*z27^20 + 8*z27^19 + 12*z27^18 + z27^16 + 9*z27^15 + 4*z27^14 + 16*z27^13 + 14*z27^12 + 7*z27^11 + 15*z27^10 + 10*z27^9 + 7*z27^8 + 2*z27^7 + 10*z27^6 + 10*z27^5 + 2*z27^3 + 13*z27^2 + 8*z27 + 2)*z^3 + 4*z27^26 + 11*z27^25 + 16*z27^23 + 12*z27^22 + 9*z27^20 + 16*z27^18 + 12*z27^17 + 11*z27^16 + z27^15 + 16*z27^14 + 14*z27^13 + 15*z27^12 + 13*z27^11 + 9*z27^10 + 14*z27^8 + 13*z27^7 + 7*z27^6 + 10*z27^5 + 15*z27^4 + 12*z27^3 + 12*z27^2 + 10*z27 + 13

In [102]:
frob_l(i3, l^2) == i3, frob_r(i3, 2) == i3

(True, True)

In [110]:
i3^(l^2)

4*z^3 + 16

In [119]:
ti = (h^(l^2)); ti

z + 4

In [112]:
h.minpoly()

x^2 + (10*x^8 + 12*x^7 + 5*x^5 + 7*x^4 + 7*x^3 + 13*x + 8)*x + 12

In [114]:
i3.minpoly()

x^2 + (11*z27^26 + 4*z27^24 + z27^23 + 6*z27^22 + 4*z27^21 + 4*z27^20 + 7*z27^19 + 4*z27^18 + 10*z27^17 + 15*z27^16 + 8*z27^15 + 14*z27^14 + 3*z27^13 + 12*z27^12 + 12*z27^11 + 10*z27^10 + 13*z27^9 + 10*z27^8 + 14*z27^7 + 16*z27^6 + 10*z27^5 + 4*z27^4 + 16*z27^3 + 15*z27^2 + 4*z27 + 14)*x + 5

In [126]:
C2([k(x) for x in (i3).list()]).polynomial().subs(C2.gen()^(n/m))

z + 4

In [122]:
h.parent() == i3.parent()

False

In [47]:
a = R([x for x in (h^(l^2)).list()]).subs(C2.gen()^l); a

NameError: name 'h' is not defined

In [156]:
b = C2([k(x) for x in (i3^(l^2)).list()]); b

4*z^3 + 16

In [159]:
t = a/b; t

13

In [162]:
rt = t.nth_root(l); rt

4

In [167]:
rnorm(4*h3, l^2)

27 9 6 2 3


(3*z27^26 + 16*z27^25 + 6*z27^24 + 7*z27^23 + 11*z27^22 + 6*z27^21 + 16*z27^20 + 2*z27^19 + 3*z27^18 + 13*z27^16 + 15*z27^15 + z27^14 + 4*z27^13 + 12*z27^12 + 6*z27^11 + 8*z27^10 + 11*z27^9 + 6*z27^8 + 9*z27^7 + 11*z27^6 + 11*z27^5 + 9*z27^3 + 16*z27^2 + 2*z27 + 9)*z^3 + z27^26 + 7*z27^25 + 4*z27^23 + 3*z27^22 + 15*z27^20 + 4*z27^18 + 3*z27^17 + 7*z27^16 + 13*z27^15 + 4*z27^14 + 12*z27^13 + 8*z27^12 + 16*z27^11 + 15*z27^10 + 12*z27^8 + 16*z27^7 + 6*z27^6 + 11*z27^5 + 8*z27^4 + 3*z27^3 + 3*z27^2 + 11*z27 + 16

In [168]:
rnorm(4*h3, l^2) == 13*rnorm(h3, l^2)

27 9 6 2 3
27 9 6 2 3


True

In [173]:
elem = change_basis(rnorm(4*h3, l^2), A3.gen()^l, 2); elem

27 9 6 2 3


z27^26 + 7*z27^25 + 4*z27^23 + 3*z27^22 + 15*z27^20 + 4*z27^18 + 3*z27^17 + 7*z27^16 + 13*z27^15 + 4*z27^14 + 12*z27^13 + 8*z27^12 + 16*z27^11 + 15*z27^10 + 12*z27^8 + 16*z27^7 + 6*z27^6 + 11*z27^5 + 8*z27^4 + 3*z27^3 + 3*z27^2 + 11*z27 + 16

In [174]:
elem.minpoly()

x^9 + 8*x^7 + 10*x^5 + 4*x^3 + 9*x + 4

In [176]:
h.list()[0].minpoly()

x^9 + 8*x^7 + 10*x^5 + 4*x^3 + 9*x + 4

# An algorithm

In [34]:
def testl(p, l, n):
    k = GF(p)
    k1, A1 = ext_scalars(k, l^n)
    M1 = A1.modulus()
    R = k['z']
    k2 = k.extension(l^(n+1))
    R2 = k2['z']
    x = R.gen()
    M2 = gcd(A1.modulus().change_ring(k).substitute(x^l), R(cyclotomic_polynomial(l^(n+1)))).factor()[0][0]
    A2 = R2.quo(M2, 'z')
    h1 = solve_h90(A1)
    h2 = solve_h90(A2)
    i2 = rnorm(h2, l^n)
    aa = h1^(l^n)
    bb = i2^(l^n)
    C = k.extension(M2, 'x')
    
    a = R([x for x in aa.list()]).subs(C.gen()^l)
    b = C([k(x) for x in bb.list()])
    
    t = a/b
    c = t.nth_root(l^(n+1))
    return h1, A2(c.polynomial().list())*h2

In [96]:
def is_iso(h1, h2):
    l = h2.parent().degree() // h1.parent().degree()
    i2 = rnorm(h2, h1.base_ring().degree())
    j2 = change_basis(i2, h2.parent().gen()^l, h1.parent().degree())
    return h1.list()[0].minpoly() == j2.minpoly()
    

In [15]:
p, l, n = 11, 3, 2

In [360]:
h1.base_ring().degree()

5

In [18]:
h1, h2 = testl(p, l, n)
is_iso(h1, h2)

True

In [356]:
is_iso(h1, h2)

True

In [290]:
C.<w> = k.extension(m2)

In [293]:
(w^l).minpoly()

x^2 + x + 1

In [363]:
for i in range(10):
    h1, h2 = testl(p, l, n)
    print is_iso(h1, h2)

True
True
True
True
True
True
True
True
True
True


In [1]:
factor(17^2-1)

2^5 * 3^2

In [47]:
t = toto(GF(19), 1, 7)[0]; t

2613660

In [7]:
cyclo_deg(GF(19), 7)

6

In [12]:
p = 19
K, A = ext_scalars(GF(p), 21)

In [16]:
z = A.gen()

In [21]:
(z^(-6))^

1

In [12]:
def frob_rc(a, n, m, i=1):
    A = a.parent()
    z = A.gen()
    l = n // m
    u, v = xgcd(l, m)[1:]
    q = a.base_ring().base_ring().order()
    return sum(c*z^(u*l*j)*z^(v*m*j*q^i) for j, c in enumerate(a))

def rnormc(a, m):
    A = a.parent()
    n = a.base_ring().degree()
    q = A.base_ring().base_ring().order()
    l = n // m
    d = cyclo_deg(GF(q), l)
    print d, l
    return product(frob_rc(a, n, m, j) for j in range(d))

In [33]:
z.base_ring().degree()

21

In [38]:
rnormc(z, 3)

6 7


1

In [51]:
q = 11
factor(q^2-1)

2^3 * 3 * 5

In [54]:
k = GF(p)

In [55]:
R.<x> = k[]

In [63]:
f = cyclotomic_polynomial(21).change_ring(k)

In [64]:
f = f.factor()[0][0]

In [65]:
f

x^6 + 7*x^5 + 11*x^4 + x^3 + 7*x^2 + 11*x + 1

In [66]:
C.<z> = k.extension(f)

In [25]:
xgcd(7, 3)[1:]

(1, -2)

In [73]:
r = C.random_element()
(r^(-2*3)).

12*z^5 + z^4 + 8*z^3 + 7*z^2 + 15*z + 4

In [72]:
K, A = ext_scalars(GF(19), 21)
h = solve_h90(A)
u = h*frob_r(h, 1)*frob_r(h, 2)*frob_r(h, 3)*frob_r(h, 4)*frob_r(h, 5)
i = rnorm(h, 1); i
# is_h90(u, A.gen()^7), frob_r(u, 21) == u, u == v
i == u

21 1 6 1 21


False

In [71]:
u

3

In [67]:
h

(17*x^6 + 8*x^5 + 13*x^4 + 3*x^3 + x^2 + x + 6)*z^5 + (13*x^6 + 3*x^5 + 6*x^3 + 15*x^2 + 2*x + 18)*z^4 + (8*x^6 + 10*x^5 + 16*x^4 + 3*x^3 + 15*x^2 + 15*x + 14)*z^3 + (15*x^6 + 13*x^5 + 12*x^4 + 3*x^3 + 8*x^2 + 8*x + 12)*z^2 + (4*x^6 + 11*x^5 + 10*x^4 + x^3 + 5*x^2 + 11*x + 7)*z + 4*x^6 + 4*x^5 + 17*x^4 + 16*x^3 + 15*x^2 + 7*x + 7

In [60]:
i^5

9*z^5

In [61]:
h^25

5*z^9 + 13*z^8 + 6*z^6 + 8*z^5 + 13*z^4 + 13*z^3 + 17*z^2 + 14*z + 9

In [89]:
cyclo_deg(GF(11), 3^3)

18

In [73]:
z = A.gen()

In [77]:
h = z
h*frob_r(h, 1)*frob_r(h, 2)*frob_r(h, 3)*frob_r(h, 4)*frob_r(h, 5)

z

In [78]:
rnorm(z, 3)

21 3 6 1 7


z

In [79]:
frob_r(h, 6)

z

In [85]:
frob_r(h, 6)

z

In [86]:
(z^.minpoly()

x^6 + 7*x^5 + 11*x^4 + x^3 + 7*x^2 + 11*x + 1

9 3 6 2 3


((7*x^2 + x + 2)*z + 2*x^2 + 9*x + 10,
 (9*z9^8 + 2*z9^7 + 3*z9^6 + z9^5 + 10*z9^4 + 5*z9^3 + 4*z9^2 + 5*z9 + 1)*z^5 + (10*z9^8 + 9*z9^7 + z9^6 + 3*z9^5 + 10*z9^4 + 6*z9^3 + 10*z9^2 + 7*z9 + 7)*z^4 + (6*z9^8 + 6*z9^7 + 7*z9^6 + 7*z9^5 + 2*z9^4 + 7*z9^3 + 3*z9^2 + 6*z9 + 4)*z^3 + (2*z9^8 + 6*z9^7 + 10*z9^6 + 6*z9^5 + 4*z9^4 + 7*z9^3 + 7*z9^2 + 3*z9)*z^2 + (4*z9^8 + 3*z9^7 + 5*z9^6 + z9^5 + 8*z9^4 + 2*z9^3 + 9*z9 + 3)*z + 5*z9^8 + 7*z9^7 + z9^6 + 6*z9^5 + 10*z9^4 + 6*z9^3 + z9^2 + 10)

In [98]:
is_iso(h1, h2)

True

4

In [94]:
l = 4

In [119]:
"""
Here we find an element h2 solution of H90 in F_p^n compatible with h1.
"""
def findh90(h1, n):
    k = h1.base_ring().base_ring()
    k1, A1 = h1.base_ring(), h1.parent()
    M1 = A1.modulus()
    R = k['z']
    m = k1.degree()
    l = n // m
    k2 = k.extension(n)
    R2 = k2['z']
    x = R.gen()
    M2 = gcd(M1.change_ring(k).substitute(x^l), R(cyclotomic_polynomial(n))).factor()[0][0]
    A2 = R2.quo(M2, 'z')
    h2 = solve_h90(A2)
    i2 = rnorm(h2, m)
    aa = h1^(m)
    bb = i2^(m)
    C = k.extension(M2, 'x')
    
    a = R([x for x in aa.list()]).subs(C.gen()^l)
    b = C([k(x) for x in bb.list()])
    
    t = a/b
    c = t.nth_root(n)
    return A2(c.polynomial().list())*h2

In [101]:
h1.base_ring()

Finite Field in x of size 11^9

In [120]:
h1, h2 = testl(11, 3, 1)

In [121]:
h2b = findh90(h1, 3^2)

In [122]:
h2 == h2b

False

In [123]:
h2

(5*z9^8 + 8*z9^7 + 4*z9^6 + z9^5 + 3*z9^3 + 2*z9^2 + 3*z9 + 6)*z^5 + (3*z9^7 + 3*z9^6 + 10*z9^5 + 6*z9^4 + 8*z9^3 + 4*z9^2 + z9 + 10)*z^4 + (5*z9^8 + z9^7 + 8*z9^6 + 6*z9^5 + 3*z9^4 + 4*z9^3 + 8*z9^2 + 6*z9 + 1)*z^3 + (9*z9^8 + z9^7 + 5*z9^6 + 6*z9^5 + 5*z9^4 + 2*z9^3 + 10*z9^2 + 4*z9 + 5)*z^2 + (6*z9^8 + 4*z9^6 + z9^5 + 7*z9^4 + z9^3 + 10*z9^2 + 5*z9 + 6)*z + 10*z9^8 + 9*z9^7 + 10*z9^6 + 4*z9^5 + 10*z9^4 + 10*z9^3 + 6*z9^2 + 8*z9 + 7

In [124]:
h2b

(4*z9^8 + 7*z9^7 + 8*z9^6 + 9*z9^5 + 2*z9^4 + 8*z9^3 + 7*z9^2 + 4*z9 + 9)*z^5 + (9*z9^7 + 4*z9^5 + 8*z9^4 + 3*z9^3 + 3*z9^2 + 7*z9 + 8)*z^4 + (7*z9^8 + 4*z9^7 + 6*z9^6 + 4*z9^5 + 8*z9^4 + 6*z9 + 2)*z^3 + (10*z9^8 + 8*z9^7 + 10*z9^6 + 5*z9^5 + 4*z9^4 + 10*z9^3 + 6*z9^2 + z9)*z^2 + (6*z9^8 + 5*z9^7 + 10*z9^6 + 7*z9^5 + 4*z9^4 + 4*z9^3 + 9*z9^2 + 8)*z + 3*z9^8 + 8*z9^5 + 6*z9^4 + 10*z9^3 + 4*z9^2 + 2*z9 + 3

In [125]:
is_iso(h1, h2)

True

In [126]:
is_iso(h1, h2b)

True

In [113]:
i2 = rnorm(h2b, 3); i2

(4*z9^8 + 2*z9^7 + 4*z9^6 + 2*z9^4 + 3*z9^3 + 10*z9^2 + 2*z9 + 7)*z^3 + 9*z9^8 + 6*z9^7 + 2*z9^6 + 8*z9^5 + 4*z9^4 + 7*z9^3 + 3*z9^2 + 7

In [114]:
i2b = rnorm(h2, 3); i2b

(3*z9^8 + 4*z9^7 + 6*z9^6 + 6*z9^5 + 8*z9^4 + 10*z9^3 + 8*z9^2 + 5*z9 + 9)*z^3 + 7*z9^8 + 7*z9^7 + 9*z9^6 + 4*z9^5 + 6*z9^4 + 4*z9^3 + 5*z9^2 + 4*z9 + 1

In [115]:
i2^3

10*z^3 + 10

In [118]:
i2b^3

7*z^3 + 5

In [130]:
h3 = findh90(h2b, 3^3)

In [131]:
is_iso(h1, h3)

True

In [132]:
rnorm(h2, 3)

(10*z9^8 + 3*z9^7 + z9^6 + 4*z9^5 + 2*z9^4 + 9*z9^3 + 7*z9^2 + 9)*z^3 + 8*z9^8 + 2*z9^7 + 10*z9^6 + 4*z9^5 + z9^4 + 2*z9^3 + 2*z9^2 + 10*z9

In [133]:
rnorm(h2b, 3)

(9*z9^8 + 10*z9^7 + 9*z9^6 + 10*z9^4 + 4*z9^3 + 6*z9^2 + 10*z9 + 2)*z^3 + z9^8 + 8*z9^7 + 10*z9^6 + 7*z9^5 + 9*z9^4 + 2*z9^3 + 4*z9^2 + 2

In [134]:
def is_iso2(h1, h2):
    l = h2.parent().degree() // h1.parent().degree()
    i2 = rnorm(h2, h1.base_ring().degree())
    j2 = change_basis(i2, h2.parent().gen()^l, h1.parent().degree())
    return h1.list()[0], j2
    

In [135]:
is_iso2(h1, h2)

(9*x, 8*z9^8 + 2*z9^7 + 10*z9^6 + 4*z9^5 + z9^4 + 2*z9^3 + 2*z9^2 + 10*z9)

In [136]:
is_iso2(h1, h2b)

(9*x, z9^8 + 8*z9^7 + 10*z9^6 + 7*z9^5 + 9*z9^4 + 2*z9^3 + 4*z9^2 + 2)